In [24]:
# setup library imports
import io, time, json
import requests
from bs4 import BeautifulSoup
import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
import re
import random

# import yelp client library
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator


In [16]:
def authenticate(config_filepath):
    """
    Create an authenticated yelp-python client.

    Args:
        config_filepath (string): relative path (from this file) to a file with your Yelp credentials

    Returns:
        client (yelp.client.Client): authenticated instance of a yelp.Client
    """
    
    creds = json.load(open(config_filepath))
    auth = Oauth1Authenticator(**creds)
    return Client(auth)

In [17]:
def yelp_search(client, query):
    """
    Make an authenticated request to the Yelp API.

    Args:
        query (string): Search term

    Returns:
        total (integer): total number of businesses on Yelp corresponding to the query
        businesses (list): list of yelp.obj.business.Business objects
    """
    
    # Write solution here
    res = client.search(query)
    return (res.total, res.businesses)

In [18]:
def all_restaurants(client, query, food):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of yelp.obj.business.Business objects
    """
    B = []
    while 1:
        try:
            res = client.search(query,**{"category_filter":food,"offset" : len(B)})
            if not res.businesses:
                return B
            B += res.businesses
#            time.sleep(.1)
        except:
            return B

In [25]:
def get_cities_info(client, food, cities):
    d = {}
    for city in cities:
        print "requesting: ", city,
        d["%s_%s"%(city,food)] = all_restaurants(client,city,food)
        print ": done"
    #d = {("%s_%s"%(city,food)):all_restaurants(client,city,food) for city in cities}
    valids = filter(lambda x: len(d[x]) >= 10, d)
    res = {v:d[v] for v in valids}
    return res

def get_all_cities():
    with open("cities.txt",'r') as f:
        lines = map(lambda l: ", ".join(l.rstrip().split("\t")[1:3]),f.readlines())
        return lines
    
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
def save_pickle(filename, data):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
        
def get_city_data(data, city, foodtype):
    '''returns dictionary:
        avg_review
        num_restaurants
        city
    '''
    result = {}
    key = "%s_%s"%(city,foodtype)
    info = data[key]
    result["num_restaurants"] = len(info)
    result["avg_review"] = sum(map(lambda x: x.rating, info))/float(len(info))
    return result
    
def get_all_city_data(data):
    result = {}
    for city_food in data:
        city = city_food.split("_")[0]
        foodtype = city_food.split("_")[1]
        result[city] = get_city_data(data,city,foodtype)
    return result

def clean_business(b):
    valids = ["categories", "location", "name", "rating", "review_count","url"]
    return {x : b.__dict__[x] for x in valids}
    
    
def clean_data(data):
    new_data = {}
    for city in data:
        new_data[city] = map(clean_business, data[city])
    return new_data

def add_feature_info(data, offset):
    c = offset+1
    for d in data[offset:]:
        print "%d of %d"%(c,len(data))
        features = scrapeYelpFeatures(d["url"])
        d.update(features)
        c += 1
        
def get_feature_set(datalist):
    features = set()
    for f in datalist:
        features |= set(f)
    return features

def fix_features(datalist):
    features = get_feature_set(datalist)
    for d in datalist:
        for f in features:
            if f not in d:
                d[f] = None

def get_random_data(data, n):
    datalist = []
    for d in data:
        datalist += data[d]
    return random.sample(datalist,n)

In [20]:
def retrieve_html(url):
    html = requests.get(url)
    return (html.status_code, html.content)

# Gets restaurant features from yelp html given a url for a restaurants page
# returns a dictionary of the available features on the page (keys) with their 
# corresponding values
def scrapeYelpFeatures(url):
    (yolo, html) = retrieve_html(url)
    
    try:
        imgs =  re.search(r"See all [0-9]+", html).group()
        imgs = [int(s) for s in imgs.split() if s.isdigit()][0]
    except:
        imgs = 0
    root = BeautifulSoup(html, "html.parser")
    s = root.findAll("div", {'class' : 'ywidget'}) #, { "class" : "review review--with-sidebar" })
    #s = root.find("h3", {'name': 'More business info'})
    
    biz_features = {"number_of_images" : imgs}
    for i in s:
        h3_elem = i.find("h3")
        if h3_elem != None:
            if 'More business info' in h3_elem:
                biz_attr = i.find('ul', {'class' : 'ylist'}).findAll('dt', {'class' : 'attribute-key'})
                biz_val = i.find('ul', {'class' : 'ylist'}).findAll('dd')

                for (l,r) in zip(biz_attr, biz_val):
                    #print l.next_element.strip(), '====', r.next_element.strip()
                    biz_features[l.next_element.strip()] = r.next_element.strip()
    return biz_features

In [14]:
info = load_pickle("Anaheim, California features.pkl")

In [ ]:
client = authenticate("config_secret.json")

In [98]:
#datalist = data['Anaheim, California'][:]
#print datalist[999]
#add_feature_info(datalist,268)
#save_pickle("Anaheim, California features.pkl", datalist)

In [104]:
#save_pickle("Anaheim, California features.pkl", datalist)

In [22]:
data = load_pickle("cities_all_restaurants_new.pkl")

In [30]:
datalist = get_random_data(data,50000)[:]

In [38]:
add_feature_info(datalist,2625)

963 of 50000
964 of 50000
965 of 50000
966 of 50000
967 of 50000
968 of 50000
969 of 50000
970 of 50000
971 of 50000
972 of 50000
973 of 50000
974 of 50000
975 of 50000
976 of 50000
977 of 50000
978 of 50000
979 of 50000
980 of 50000
981 of 50000
982 of 50000
983 of 50000
984 of 50000
985 of 50000
986 of 50000
987 of 50000
988 of 50000
989 of 50000
990 of 50000
991 of 50000
992 of 50000
993 of 50000
994 of 50000
995 of 50000
996 of 50000
997 of 50000
998 of 50000
999 of 50000
1000 of 50000
1001 of 50000
1002 of 50000
1003 of 50000
1004 of 50000
1005 of 50000
1006 of 50000
1007 of 50000
1008 of 50000
1009 of 50000
1010 of 50000
1011 of 50000
1012 of 50000
1013 of 50000
1014 of 50000
1015 of 50000
1016 of 50000
1017 of 50000
1018 of 50000
1019 of 50000
1020 of 50000
1021 of 50000
1022 of 50000
1023 of 50000
1024 of 50000
1025 of 50000
1026 of 50000
1027 of 50000
1028 of 50000
1029 of 50000
1030 of 50000
1031 of 50000
1032 of 50000
1033 of 50000
1034 of 50000
1035 of 50000
1036 of 50000
1

KeyboardInterrupt: 

In [39]:
done = datalist[:2625]

In [40]:
kevin = load_pickle("kevin_processed.pkl")

In [50]:
save_pickle("random.pkl", done)